In [36]:

import findspark
from datetime import datetime
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, udf, col, when
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType,FloatType, DoubleType
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

23/06/22 20:35:40 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-31a483cf-4d30-4f19-a61e-36d08e9d3fb9--2001384727-driver-0-1, groupId=spark-kafka-source-31a483cf-4d30-4f19-a61e-36d08e9d3fb9--2001384727-driver-0] Seeking to LATEST offset of partition ticketmaster-0
23/06/22 20:35:40 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-0f361c7e-4cd5-4e48-8290-a06ff9a0332a--2042041463-driver-0-3, groupId=spark-kafka-source-0f361c7e-4cd5-4e48-8290-a06ff9a0332a--2042041463-driver-0] Seeking to LATEST offset of partition ticketmaster-0
23/06/22 20:35:40 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-31a483cf-4d30-4f19-a61e-36d08e9d3fb9--2001384727-driver-0-1, groupId=spark-kafka-source-31a483cf-4d30-4f19-a61e-36d08e9d3fb9--2001384727-driver-0] Resetting offset for partition ticketmaster-0 to offset 1369.
23/06/22 20:35:40 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-0f361c7e-4cd5-4e48-8290-a06ff9a0332

In [17]:
findspark.init()
topic_name = 'ticketmaster'

spark_path = findspark.find()
print(spark_path)



/Users/nicburkett/.sdkman/candidates/spark/current


In [ ]:
credentials_location = '/Users/nicburkett/.google/credentials/google_credentials.json'

# Spark COnfig
conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('TwitterSentimentAnalysis') \
    .set("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .set("spark.jars", "gs://path/to/spark-bigquery-latest.jar,gs://path/to/google-cloud-bigquery-latest.jar,/path/to/local-jar-file.jar") \
    .set("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)\
    .set("spark.jars", "gcs-connector-hadoop3-2.2.5.jar") 


sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

# Spark Context
# sc = spark.sparkContext
# sc.setLogLevel('ERROR')

spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [50]:
# schema = StructType([
#     StructField("event_name", StringType(), nullable=True),
#     StructField("event_type", StringType(), nullable=True),
#     StructField("event_id", StringType(), nullable=True),
#     StructField("event_url", StringType(), nullable=True),
#     StructField("venue_name", StringType(), nullable=True),
#     StructField("venue_id", StringType(), nullable=True),
#     StructField("venue_zipcode", IntegerType(), nullable=True),
#     StructField("venues_timezone", StringType(), nullable=True),
#     StructField("venue_city", StringType(), nullable=True),
#     StructField("venue_state_full", StringType(), nullable=True),
#     StructField("venue_state_short", StringType(), nullable=True),
#     StructField("venue_country_name", StringType(), nullable=True),
#     StructField("venue_country_short", StringType(), nullable=True),
#     StructField("venue_address", StringType(), nullable=True),
#     StructField("venue_longitude", DoubleType(), nullable=True),
#     StructField("venue_latitude", DoubleType(), nullable=True),
#     StructField("attraction_name", StringType(), nullable=True),
#     StructField("attraction_type", StringType(), nullable=True),
#     StructField("attraction_id", StringType(), nullable=True),
#     StructField("attraction_url", StringType(), nullable=True),
#     StructField("attraction_segment_id", StringType(), nullable=True),
#     StructField("attraction_segment_name", StringType(), nullable=True),
#     StructField("attraction_genre_id", StringType(), nullable=True),
#     StructField("attraction_genre_name", StringType(), nullable=True),
#     StructField("attraction_subgenre_id", StringType(), nullable=True),
#     StructField("attraction_subgenre_name", StringType(), nullable=True),
#     StructField("event_start_date", DateType(), nullable=True),
#     StructField("ticket_status", StringType(), nullable=True),
#     StructField("event_start_time", StringType(), nullable=True),
#     StructField("currency", StringType(), nullable=True),
#     StructField("min_price", FloatType(), nullable=True),
#     StructField("max_price", FloatType(), nullable=True)
# ])





from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

schema = StructType([
    StructField("event_name", StringType()),
    StructField("event_type", StringType()),
    StructField("event_id", StringType()),
    StructField("event_url", StringType()),
    StructField("venue_name", StringType()),
    StructField("venue_id", StringType()),
    StructField("venue_zipcode", StringType()),
    StructField("venues_timezone", StringType()),
    StructField("venue_city", StringType()),
    StructField("venue_state_full", StringType()),
    StructField("venue_state_short", StringType()),
    StructField("venue_country_name", StringType()),
    StructField("venue_country_short", StringType()),
    StructField("venue_address", StringType()),
    StructField("venue_longitude", StringType()),
    StructField("venue_latitude", StringType()),
    StructField("attraction_name", StringType()),
    StructField("attraction_type", StringType()),
    StructField("attraction_id", StringType()),
    StructField("attraction_url", StringType()),
    StructField("attraction_segment_id", StringType()),
    StructField("attraction_segment_name", StringType()),
    StructField("attraction_genre_id", StringType()),
    StructField("attraction_genre_name", StringType()),
    StructField("attraction_subgenre_id", StringType()),
    StructField("attraction_subgenre_name", StringType()),
    StructField("event_start_date", StringType()),
    StructField("ticket_status", StringType()),
    StructField("event_start_time", StringType()),
    StructField("currency", StringType()),
    StructField("min_price", StringType()),
    StructField("max_price", StringType())
])





23/06/22 20:57:40 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-2b8cbc25-4bf7-465c-b61f-6e26ba34a2b9--2109907342-driver-0-33, groupId=spark-kafka-source-2b8cbc25-4bf7-465c-b61f-6e26ba34a2b9--2109907342-driver-0] Seeking to LATEST offset of partition ticketmaster-0
23/06/22 20:57:40 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-8c615a1e-fabc-43c7-ba3c-87f1c674f1d4--2001384727-driver-0-40, groupId=spark-kafka-source-8c615a1e-fabc-43c7-ba3c-87f1c674f1d4--2001384727-driver-0] Seeking to LATEST offset of partition ticketmaster-0
23/06/22 20:57:40 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-8c615a1e-fabc-43c7-ba3c-87f1c674f1d4--2001384727-driver-0-40, groupId=spark-kafka-source-8c615a1e-fabc-43c7-ba3c-87f1c674f1d4--2001384727-driver-0] Resetting offset for partition ticketmaster-0 to offset 1429.
23/06/22 20:57:40 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-2b8cbc25-4bf7-465c-b61f-6e26ba34

In [20]:

PROJECT_ID = 'global-maxim-338114'
BUCKET = "kafka-spark-data"

dataset = 'global-maxim-338114.twitter_kafka_pyspark_test'
table = 'twitter_kafka_pyspark_test'

gcs_metadata_folder = "gs://kafka-spark-data/spark-metadata"
gcs_data_folder = "gs://kafka-spark-data/raw-spark-data"
print(PROJECT_ID)

global-maxim-338114


In [ ]:
# CHECK DATA TYPES 
## BATCH DATA PROCESSING 

df = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", topic_name) \
  .load()\
  .selectExpr("CAST(value AS STRING)")


# Apply deserialization or further processing if needed
df1 = df.withColumn("parsed_data", from_json("value",))

# Extract and convert the "venue_zipcode" as Integer
df1 = df1.withColumn("venue_zipcode", col("parsed_data.venue_zipcode").cast(IntegerType()))
# Extract and convert the coordinates as Double
df1 = df1.withColumn("venue_longitude", col("parsed_data.venue_longitude").cast(DoubleType()))
df1 = df1.withColumn("venue_latitude", col("parsed_data.venue_latitude").cast(DoubleType()))
# Extract and Convert the event_start_date as Date 
df1 = df1.withColumn("event_start_date", col("parsed_data.event_start_date").cast(DateType()))
# Extract and Convert the event_start_date as Date 
# df1 = df1.withColumn("event_start_time", col("parsed_data.event_start_date").cast(TimeType()))


# Select the desired columns
df2 = df1.select("parsed_data.*")


df2.printSchema()
df2.show(5)

In [56]:
##### STREAMING DATA PROCESSING #####

# Read the data from kafka
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", topic_name) \
    .option("startingOffsets", "latest") \
    .option("header", "true") \
    .load() \
    .selectExpr("CAST(value AS STRING)")
    
# Apply deserialization or further processing if needed
df1 = df.withColumn("parsed_data", from_json("value", schema))

### DATA TYPE CONVERSIONS ####
# Extract and convert the "venue_zipcode" as Integer
# df1 = df1.withColumn("venue_zipcode", col("parsed_data.venue_zipcode").cast(IntegerType()))
# # Extract and convert the coordinates as Double
# df1 = df1.withColumn("venue_longitude", col("parsed_data.venue_longitude").cast(DoubleType()))
# df1 = df1.withColumn("venue_latitude", col("parsed_data.venue_latitude").cast(DoubleType()))
# # Extract and Convert the event_start_date as Date 
# df1 = df1.withColumn("event_start_date", col("parsed_data.event_start_date").cast(DateType()))

df2 = df1.select("parsed_data.*")


df2.printSchema()

path = '/Users/nicburkett/Desktop/spark_output'


# # # Write to a local file
# # file_query = df2.writeStream \
# #     .format("csv") \
# #     .outputMode("append") \
# #     .option("header", "true") \
# #     .option("checkpointLocation", path) \
# #     .trigger(processingTime="10 seconds") \
# #     .start(path)

# # # WRITE TO GCS BUCKET 
# gcs_write = df2.writeStream \
#     .format("csv") \
#     .outputMode("append") \
#     .option("path","gs://kafka-spark-data/raw-spark-data") \
#     .option("checkpointLocation", "gs://kafka-spark-data/spark-metadata") \
#     .trigger(processingTime="10 seconds") \
#     .start() 

# gcs_write.awaitTermination()

# WRITE TO CONSOLE TO LOG 
# console_query = df2.writeStream \
#     .format("console") \
#     .outputMode("append") \
#     .trigger(processingTime="10 seconds") \
#     .start() \
#     .awaitTermination()

    # .foreachBatch(write_batch) \

gcs_bigquery_stream = df2.writeStream \
    .format("bigquery") \
    .trigger(processingTime="10 seconds") \
    .option("checkpointLocation", gcs_metadata_folder) \
    .option("temporaryGcsBucket", 'kafka-spark-data') \
    .option("table",'global-maxim-338114.twitter_kafka_pyspark_test.kafka_pyspark') \
    .option("mode", "FAILFAST") \
    .start()

    # .option("failOnDataLoss",'false') \

gcs_bigquery_stream.awaitTermination()

root
 |-- event_name: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- event_id: string (nullable = true)
 |-- event_url: string (nullable = true)
 |-- venue_name: string (nullable = true)
 |-- venue_id: string (nullable = true)
 |-- venue_zipcode: string (nullable = true)
 |-- venues_timezone: string (nullable = true)
 |-- venue_city: string (nullable = true)
 |-- venue_state_full: string (nullable = true)
 |-- venue_state_short: string (nullable = true)
 |-- venue_country_name: string (nullable = true)
 |-- venue_country_short: string (nullable = true)
 |-- venue_address: string (nullable = true)
 |-- venue_longitude: string (nullable = true)
 |-- venue_latitude: string (nullable = true)
 |-- attraction_name: string (nullable = true)
 |-- attraction_type: string (nullable = true)
 |-- attraction_id: string (nullable = true)
 |-- attraction_url: string (nullable = true)
 |-- attraction_segment_id: string (nullable = true)
 |-- attraction_segment_name: string (nu

23/06/22 21:39:03 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687482506437-1c296931-c548-4a86-ac0b-a9c16ec7b8fa/_temporary/0/_temporary/' directory.
23/06/22 21:39:03 INFO FileOutputCommitter: Saved output of task 'attempt_202306222139018453926043173608051_0222_m_000000_221' to gs://kafka-spark-data/.spark-bigquery-local-1687482506437-1c296931-c548-4a86-ac0b-a9c16ec7b8fa/_temporary/0/task_202306222139018453926043173608051_0222_m_000000
23/06/22 21:39:03 INFO SparkHadoopMapRedUtil: attempt_202306222139018453926043173608051_0222_m_000000_221: Committed. Elapsed time: 918 ms.
23/06/22 21:39:03 INFO Executor: Finished task 0.0 in stage 222.0 (TID 221). 2579 bytes result sent to driver
23/06/22 21:39:03 INFO TaskSetManager: Finished task 0.0 in stage 222.0 (TID 221) in 1701 ms on 192.168.4.24 (executor driver) (1/1)
23/06/22 21:39:03 INFO TaskSchedulerImpl: Removed TaskSet 222.0, whose tasks have all completed, from pool 
23/06/22 2

23/06/22 21:39:04 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687482506437-1c296931-c548-4a86-ac0b-a9c16ec7b8fa/_temporary/0/task_202306222139018453926043173608051_0222_m_000000/' directory.
23/06/22 21:39:05 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687482506437-1c296931-c548-4a86-ac0b-a9c16ec7b8fa/' directory.
23/06/22 21:39:05 INFO FileFormatWriter: Write Job 529209d2-ab96-4545-9dad-a3eb556d548d committed. Elapsed time: 2166 ms.
23/06/22 21:39:05 INFO FileFormatWriter: Finished processing stats for write job 529209d2-ab96-4545-9dad-a3eb556d548d.
23/06/22 21:39:06 INFO BigQueryClient: Submitted job LoadJobConfiguration{type=LOAD, destinationTable=GenericData{classInfo=[datasetId, projectId, tableId], {datasetId=twitter_kafka_pyspark_test, projectId=global-maxim-338114, tableId=kafka_pyspark}}, decimalTargetTypes=null, destinationEncryptionConfiguration=null, createD

23/06/22 21:39:14 INFO KafkaConsumer: [Consumer clientId=consumer-spark-kafka-source-2dddb316-bacd-4c13-8f6c-9f3fb9aad032--2001384727-executor-55, groupId=spark-kafka-source-2dddb316-bacd-4c13-8f6c-9f3fb9aad032--2001384727-executor] Seeking to offset 1686 for partition ticketmaster-0
23/06/22 21:39:14 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-2dddb316-bacd-4c13-8f6c-9f3fb9aad032--2001384727-executor-55, groupId=spark-kafka-source-2dddb316-bacd-4c13-8f6c-9f3fb9aad032--2001384727-executor] Seeking to EARLIEST offset of partition ticketmaster-0
23/06/22 21:39:14 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-2dddb316-bacd-4c13-8f6c-9f3fb9aad032--2001384727-executor-55, groupId=spark-kafka-source-2dddb316-bacd-4c13-8f6c-9f3fb9aad032--2001384727-executor] Resetting offset for partition ticketmaster-0 to offset 0.
23/06/22 21:39:14 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-2dddb316-bacd-4c13-8f6c-9f3fb9aad032--

23/06/22 21:39:16 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687482506437-4b5e1a4d-0940-4aaf-8f04-151108e919c5/_temporary/0/task_20230622213913101829703990907872_0224_m_000000/' directory.
23/06/22 21:39:17 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687482506437-4b5e1a4d-0940-4aaf-8f04-151108e919c5/' directory.
23/06/22 21:39:17 INFO FileFormatWriter: Write Job 25aabd0f-f8e3-469a-9d84-4f7534857293 committed. Elapsed time: 1964 ms.
23/06/22 21:39:17 INFO FileFormatWriter: Finished processing stats for write job 25aabd0f-f8e3-469a-9d84-4f7534857293.
23/06/22 21:39:18 INFO BigQueryClient: Submitted job LoadJobConfiguration{type=LOAD, destinationTable=GenericData{classInfo=[datasetId, projectId, tableId], {datasetId=twitter_kafka_pyspark_test, projectId=global-maxim-338114, tableId=kafka_pyspark}}, decimalTargetTypes=null, destinationEncryptionConfiguration=null, createDi

...

23/06/22 21:47:34 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-2dddb316-bacd-4c13-8f6c-9f3fb9aad032--2001384727-executor-55, groupId=spark-kafka-source-2dddb316-bacd-4c13-8f6c-9f3fb9aad032--2001384727-executor] Resetting offset for partition ticketmaster-0 to offset 0.
23/06/22 21:47:34 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-2dddb316-bacd-4c13-8f6c-9f3fb9aad032--2001384727-executor-55, groupId=spark-kafka-source-2dddb316-bacd-4c13-8f6c-9f3fb9aad032--2001384727-executor] Seeking to LATEST offset of partition ticketmaster-0
23/06/22 21:47:34 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-2dddb316-bacd-4c13-8f6c-9f3fb9aad032--2001384727-executor-55, groupId=spark-kafka-source-2dddb316-bacd-4c13-8f6c-9f3fb9aad032--2001384727-executor] Resetting offset for partition ticketmaster-0 to offset 4831.
23/06/22 21:47:35 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-l

23/06/22 21:47:35 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687482506437-ae47c2fd-4ef5-4748-9901-719a536cd12b/_temporary/0/task_20230622214733973222591584667979_0272_m_000000/' directory.
23/06/22 21:47:36 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687482506437-ae47c2fd-4ef5-4748-9901-719a536cd12b/' directory.
23/06/22 21:47:36 INFO BlockManagerInfo: Removed broadcast_271_piece0 on 192.168.4.24:51040 in memory (size: 94.8 KiB, free: 434.3 MiB)
23/06/22 21:47:36 INFO BlockManagerInfo: Removed broadcast_269_piece0 on 192.168.4.24:51040 in memory (size: 94.8 KiB, free: 434.4 MiB)
23/06/22 21:47:36 INFO FileFormatWriter: Write Job a215e29f-2cd2-45af-95ba-3b780434ecf6 committed. Elapsed time: 1423 ms.
23/06/22 21:47:36 INFO FileFormatWriter: Finished processing stats for write job a215e29f-2cd2-45af-95ba-3b780434ecf6.
23/06/22 21:47:36 INFO BigQueryClient: Submitted job L

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/nicburkett/opt/anaconda3/envs/spark/lib/python3.11/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nicburkett/opt/anaconda3/envs/spark/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nicburkett/opt/anaconda3/envs/spark/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o1447.awaitTermination

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/nicburkett/opt/anaconda3/envs/spark/lib/python3.11/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/nicburkett/opt/anaconda3/envs/spark/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nicburkett/opt/anaconda3/envs/spark/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [42]:
# WRITE TO CONSOLE TO LOG 
console_query = df2.writeStream \
    .format("console") \
    .outputMode("append") \
    .trigger(processingTime="10 seconds") \
    .start() \
    .awaitTermination()

    # .foreachBatch(write_batch) \

23/06/22 20:39:55 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/gr/l9vf1gtd2_s_3tz5n15mgkn80000gn/T/temporary-7718572b-fe8e-461e-9e94-f12a490f995c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/06/22 20:39:55 INFO ResolveWriteToStream: Checkpoint root /private/var/folders/gr/l9vf1gtd2_s_3tz5n15mgkn80000gn/T/temporary-7718572b-fe8e-461e-9e94-f12a490f995c resolved to file:/private/var/folders/gr/l9vf1gtd2_s_3tz5n15mgkn80000gn/T/temporary-7718572b-fe8e-461e-9e94-f12a490f995c.
23/06/22 20:39:55 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/06/22 20:39:55 INFO CheckpointFileManager: Writing atomically to file:/private/var/folders/gr/l9vf1gtd2_s_3tz5n15mgkn80000gn/T/te

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/nicburkett/opt/anaconda3/envs/spark/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nicburkett/opt/anaconda3/envs/spark/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nicburkett/opt/anaconda3/envs/spark/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

23/06/22 20:40:20 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-2b8cbc25-4bf7-465c-b61f-6e26ba34a2b9--2109907342-driver-0-33, groupId=spark-kafka-source-2b8cbc25-4bf7-465c-b61f-6e26ba34a2b9--2109907342-driver-0] Seeking to LATEST offset of partition ticketmaster-0
23/06/22 20:40:20 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-2b8cbc25-4bf7-465c-b61f-6e26ba34a2b9--2109907342-driver-0-33, groupId=spark-kafka-source-2b8cbc25-4bf7-465c-b61f-6e26ba34a2b9--2109907342-driver-0] Resetting offset for partition ticketmaster-0 to offset 1393.
23/06/22 20:40:20 INFO MicroBatchExecution: Streaming query made progress: {
  "id" : "5ff3f0d9-8649-4084-a0de-9c984d47f474",
  "runId" : "061d6392-20e9-47dc-a164-da2a57f51557",
  "name" : null,
  "timestamp" : "2023-06-23T01:40:20.005Z",
  "batchId" : 1,
  "numInputRows" : 0,
  "inputRowsPerSecond" : 0.0,
  "processedRowsPerSecond" : 0.0,
  "durationMs" : {
    "latestOffset" : 10,
    "triggerExecution" : 

In [23]:
spark.stop()

23/06/19 19:50:27 INFO FetchSessionHandler: [Consumer clientId=consumer-spark-kafka-source-d2941ca6-4ad1-48ba-bda3-65af4267d63f--2001384727-executor-9, groupId=spark-kafka-source-d2941ca6-4ad1-48ba-bda3-65af4267d63f--2001384727-executor] Error sending fetch request (sessionId=INVALID, epoch=INITIAL) to node 1001:
org.apache.kafka.common.errors.TimeoutException: Failed to send request after 30000 ms.
23/06/19 19:50:42 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-d2941ca6-4ad1-48ba-bda3-65af4267d63f--2001384727-executor-9, groupId=spark-kafka-source-d2941ca6-4ad1-48ba-bda3-65af4267d63f--2001384727-executor] Connection to node 1001 (host.docker.internal/143.244.220.150:9092) could not be established. Broker may not be available.
23/06/19 19:50:42 INFO FetchSessionHandler: [Consumer clientId=consumer-spark-kafka-source-d2941ca6-4ad1-48ba-bda3-65af4267d63f--2001384727-executor-9, groupId=spark-kafka-source-d2941ca6-4ad1-48ba-bda3-65af4267d63f--2001384727-executor] Erro

In [ ]:

## BATCH DATA PROCESSING 

df = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", topic_name) \
  .load()\
  .selectExpr("CAST(value AS STRING)")

# Apply deserialization or further processing if needed
df1 = df.withColumn("parsed_data", from_json("value", schema))
## Select the data from the parsed_data column
df2 = df1.select("parsed_data.*")

gcs_metadata_folder = "gs://kafka-spark-data/spark-metadata"
gcs_data_folder = "gs://kafka-spark-data/raw-spark-data"

print(df2.schema)

## WRITE TO LOCAL STORAGE
# gcs_write = df2.write \
#   .format("csv") \
#   .option("checkpointLocation", "/Users/nicburkett/Desktop/spark_output") \
#   .option("path","/Users/nicburkett/Desktop/spark_output") \
#   .mode("overwrite")\
#   .save()

##WRITE TO GCS BUCKET
# gcs_write_newfolder = df2.write \
#   .format("parquet") \
#   .option("checkpointLocation", gcs_metadata_folder) \
#   .option("path",gcs_data_folder) \
#   .mode("overwrite")\
#   .save()


dataset = 'global-maxim-338114.twitter_kafka_pyspark_test'
table = 'twitter_kafka_pyspark_test'

# Write the DataFrame to BigQuery
 ## this is the bucket where the data is stored temporarily
df2.write \
    .format("bigquery") \
    .mode("overwrite") \
    .option("checkpointLocation", gcs_metadata_folder) \
    .option("temporaryGcsBucket", 'kafka-spark-data') \
    .option("table",'global-maxim-338114.twitter_kafka_pyspark_test.kafka_pyspark') \
    .option("mode", "FAILFAST") \
    .save()


In [ ]:
import pandas as pd
# Function to save the data to GCS with a custom filename
def save_to_gcs(batch_df, batch_id):
    # Convert the batch dataframe to a pandas dataframe
    pandas_df = batch_df.toPandas()

    # Get the values of the desired columns from the first row
    column1_value = pandas_df.loc[0, "column1"]
    column2_value = pandas_df.loc[0, "column2"]

    # Get the current time
    current_time = pd.Timestamp.now()

    # Generate the custom filename
    filename = f"file_{column1_value}_{column2_value}_{current_time}.parquet"

    # Save the dataframe to GCS with the custom filename
    pandas_df.to_parquet(f"gs://{bucket_name}/{path}/{filename}", index=False)


In [ ]:
# DATA TYPES THOUGHOUT KAFKA SERVER
import json
import pandas as pd
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, udf
import findspark

findspark.init()
topic_name = 'twitter'
# Config
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("TwitterSentimentAnalysis") \
    .getOrCreate()

prod = {'user_id': 19, 'recipient_id': 57, 'message': 'YbfyRHyWgjuGlzOiudEcVMLJNzqUPDvV'}
print(type(prod))
##convert dictionary to json string (BYTES)
serialized_prod = json.dumps(prod).encode('utf-8')

print(f'The producer dtype is {type(serialized_prod)} and output is {(serialized_prod)}')

## turn from string/bytes into a dictionary again
deserializer_function = lambda x: json.loads(x.decode('utf-8'))
deserialized_cons = deserializer_function(serialized_prod)

print(f'The producer dtype is {type(deserialized_cons)} and output is {(deserialized_cons)}')

### PARSING THE JSON COMING OUT 
user_id = deserialized_cons.get('user_id')
recipient_id = deserialized_cons.get('recipient_id')
message = deserialized_cons.get('message')
output_parsed = print(f'UserID: {user_id}, RecipientID: {recipient_id}, Message:{message}')


df_pandas = pd.DataFrame([deserialized_cons])
df_pandas


# df_spark = spark.createDataFrame(df_pandas)
# df_spark.show()

# Create a spark schema/column headers
schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("recipient_id", IntegerType(), True),
    StructField("message", StringType(), True)
])

# Create DataFrame from a single row
# data = [(user_id, recipient_id, message)]
df_spark = spark.createDataFrame(df_pandas,schema)
df_spark.show()
# df.write.csv('/path/to/output.csv', header=True, mode='overwrite')
